## Load the dataset and create the vocabulary

In [1]:
!pip install dvc

     |████████████████████████████████| 399 kB 5.3 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
     |████████████████████████████████| 44 kB 2.6 MB/s 
     |████████████████████████████████| 132 kB 44.9 MB/s 
     |████████████████████████████████| 41 kB 40 kB/s 
     |████████████████████████████████| 546 kB 30.2 MB/s 
     |████████████████████████████████| 4.6 MB 32.5 MB/s 
     |████████████████████████████████| 296 kB 42.2 MB/s 
     |████████████████████████████████| 109 kB 35.4 MB/s 
     |████████████████████████████████| 214 kB 43.9 MB/s 
     |████████████████████████████████| 180 kB 44.8 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 2.6 MB 49.1 MB/s 
     |████████████████████████████████| 222 kB 47.4 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |██████████████████████

In [2]:
!dvc get https://github.com/iterative/aita_dataset aita_clean.csv

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('aita_clean.csv')
df = df[df['score'] >= 10]
df['text'] = df['title'] + df['body'].fillna('')
lines = df['text']
labels = df['is_asshole']
new_df = df[['text', 'is_asshole']]
new_df.reset_index(drop=True, inplace=True) 
train_set, test_set = train_test_split(new_df, test_size=0.3)

In [7]:
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en')

In [8]:
# build the vocab
counter = Counter()
for i, line in enumerate(lines):
    counter.update(tokenizer(str(line)))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

## Create embedding vectors from GloVe

In [9]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

## Build up train/test dataset

In [40]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transform input text to vectors
def process_text(text):
    return vocab(tokenizer(text))

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_index, _text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    lengths = torch.tensor(lengths, dtype=torch.float)

    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]

    text_list = pad_sequence(text_list, batch_first=True)
    return  label_list, text_list, lengths.to(device)

train_dataset = to_map_style_dataset(train_set.itertuples())
test_dataset = to_map_style_dataset(test_set.itertuples())

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)

In [34]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transform input text to vectors
def process_text(text):
    return vocab(tokenizer(text))

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_index, _text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]

    text_list = pad_sequence(text_list, batch_first=True)
    lengths = torch.tensor(lengths, dtype=torch.float)
    return  label_list, text_list, lengths.to(device)


train_dataset = to_map_style_dataset(train_set.itertuples())
test_dataset = to_map_style_dataset(test_set.itertuples())

train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

class LSTMcustom(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        # Create a 1D-CNN use torch.nn.Conv1d and feed the output to the next LSTM layer. 
        self.cnn = torch.nn.Conv1d(in_channels=embed_dim, out_channels=20, kernel_size=2)
        # Determine the input shape of this LSTM layer.
        self.lstm = nn.LSTM(20, 200, 1, bidirectional=False, batch_first = True)

        self.fc = nn.Linear(200, 1)
                
    def forward(self, text, lengths):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)
        # Original sequence and embedding_dim can change after applying CNN, use torch.permute to transpose

        embedded = torch.permute(embedded, (0, 2, 1)) # (batch_size, emb_size, sent_len)
        
        cnn_out = self.cnn(embedded) # (batch_size, emb_size, sent_len)

        cnn_out = torch.permute(cnn_out, (0, 2, 1)) # (batch_size, sent_len, emb_size)

        lstm_out,_ = self.lstm(cnn_out) # lstm_out is a 3d tensor (batch_size, seq_len, output_size)
        
        lstm_out = lstm_out[:, -1, :]

        return torch.sigmoid(self.fc(lstm_out)) 

In [36]:
from imblearn.under_sampling import NearMiss
import numpy as np

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50

    for idx, (label, text, length) in enumerate(dataloader):
        undersample = NearMiss(version=3, n_neighbors=2)
        X_train, Y_train = undersample.fit_resample(text, label)
        lengths = []
        label_list = torch.tensor(Y_train, dtype=torch.float)[:,None]
        label = label_list.to(device)
        lengths = torch.tensor(lengths, dtype=torch.float)
        text = text.to(device)
        lengths = lengths.to(device)
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, lengths)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, lengths) in enumerate(dataloader):
            label = label.to(device)
            text = text.to(device)
            predicted_label = model(text, lengths)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [18]:
EPOCHS = 30 # epoch

model = LSTMcustom(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()
total_accu = None


for epoch in range(1, EPOCHS + 1):
    train(train_dataloader)

/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

| epoch   1 |    50/  268 batches | accuracy    0.297


/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

| epoch   1 |   100/  268 batches | accuracy    0.312


/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

...

/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

| epoch  30 |   200/  268 batches | accuracy    0.718


/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

| epoch  30 |   250/  268 batches | accuracy    0.704


/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:176: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  "The number of the samples to be selected is larger"
/usr/local

In [37]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy    92.55%


In [41]:
def acc_for_each_class(model,dataloader):
    model.eval()
    nonasshole_acc, nonasshole_count = 0 , 0
    asshole_acc, asshol_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, lengths) in enumerate(dataloader):
            label = label.to(device)
            text = text.to(device)
            predicted_label = model(text, lengths)
            correct=((predicted_label > 0.5) == label)

            for i,l in enumerate(label):
                    if(l.data[0] < 0.5):
                        nonasshole_count += 1
                        if(correct[i].data[0]):
                            nonasshole_acc += 1
                    else:
                        asshol_count += 1.0
                        if(correct[i].data[0]):
                            asshole_acc += 1
      
    return nonasshole_acc/nonasshole_count, asshole_acc/asshol_count

In [42]:
nonasshole_acc, asshole_acc = acc_for_each_class(model, test_dataloader)
print('The accuracy for the non-asshole calss is : {}'.format(nonasshole_acc))
print('The accuracy for the asshole calss is : {}'.format(asshole_acc))

The accuracy for the non-asshole calss is : 0.0
The accuracy for the asshole calss is : 1.0


In [44]:
def get_f1_score(model,dataloader,class_name):
    model.eval()
    FP = 0
    TP = 0
    FN = 0

    with torch.no_grad():
        for idx, (label, text, lengths) in enumerate(dataloader):
            label = label.to(device)
            text = text.to(device)
            predicted_label = model(text, lengths)
            correct=((predicted_label > 0.5) == label)

            for i,l in enumerate(label):
                if class_name == 'asshole':
                  if(l.data[0] > 0.5):
                      if(correct[i].data[0]):
                          TP += 1
                      else:
                          FP += 1
                  else:
                      if(not correct[i].data[0]):
                          FN += 1
                else:
                      if(l.data[0] < 0.5):
                          if(correct[i].data[0]):
                              TP += 1
                          else:
                              FP += 1
                      else:
                          if(not correct[i].data[0]):
                              FN += 1
      
    return TP/(TP + 0.5 * (FP + FN))

In [45]:
f1_score_asshole = get_f1_score(model, test_dataloader, 'asshole')
print('The f1 score for asshole class is : {}'.format(f1_score_asshole))
f1_score_nonasshole = get_f1_score(model, test_dataloader, 'nonasshole')
print('The f1 score for nonasshole class is : {}'.format(f1_score_nonasshole))

The f1 score for asshole class is : 0.43167469234884964
The f1 score for nonasshole class is : 0.0
